# SNe in host galaxies
Owner: Alex Kim <br>
Last Verified to Run: 2018-07-23

In [1]:
import numpy
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from astropy.coordinates import SkyCoord
import GCRCatalogs
import buzzard_stellar_mass
import itertools
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import SkyCoord
from astropy import units as u

In [3]:
seed=1234
controlTime = 1.e1

In [4]:
MB = -19.25
sigma_MB = 0.08
cosmo = FlatLambdaCDM(70, 0.286)

In [5]:
# load coadd catalog (for a single tract)
coadd_cat = GCRCatalogs.load_catalog('buzzard_v1.6_test')

In [6]:
quantities = ["redshift", "redshift_true","ra","dec","galaxy_id",
              "truth/COEFFS/0","truth/COEFFS/1","truth/COEFFS/2","truth/COEFFS/3","truth/COEFFS/4"]
filts = [
    'redshift < 0.2',
    (numpy.isfinite,"truth/COEFFS/0"),
    (numpy.isfinite,"truth/COEFFS/1"),
    (numpy.isfinite,"truth/COEFFS/2"),
    (numpy.isfinite,"truth/COEFFS/3"),
    (numpy.isfinite,"truth/COEFFS/4"),
    ]

In [7]:
kfile = "/global/projecta/projectdirs/lsst/groups/CS/Buzzard/metadata/templates/k_nmf_derived.default.fits"
gdq = buzzard_stellar_mass.GetDerivedQuantities(kfile)

In [8]:
out = None

In [9]:
numpy.random.seed(seed)

In [10]:
for data in coadd_cat.get_quantities(quantities, filters = filts, return_iterator=True):
    sfr_met_smass=gdq.get_derived_quantities(numpy.array([data["truth/COEFFS/0"],data["truth/COEFFS/1"],
                                              data["truth/COEFFS/2"],data["truth/COEFFS/3"],
                                              data["truth/COEFFS/4"]]).T,data["redshift_true"])
    rate = controlTime*(1.23e-03*sfr_met_smass[0]**(.95)+.26e-10*sfr_met_smass[2]**(.72)/(1+data["redshift_true"]))
    nsne = numpy.random.poisson(rate)
    w = nsne > 0
    for key, value in data.items():
        data[key]=value[w]
    data['nsne'] = nsne[w]
    data['nsne'][1]=2
    data['mB']  = []
    for nsn, redtrue in zip(data['nsne'], data["redshift_true"]):
        data['mB'].append(MB + numpy.random.normal(scale=sigma_MB,size=nsn)+cosmo.distmod(redtrue).value)
#    data['mB'] = MB + numpy.random.normal(scale=sigma_MB,size=len(data["redshift_true"])) + \
#        cosmo.distmod(data["redshift_true"]).value  
    data['mB_expected'] = MB + cosmo.distmod(data["redshift"]).value
    c = SkyCoord(ra=data["ra"]*u.degree, dec=data["dec"]*u.degree, frame='icrs')
    data['l'] = c.galactic.spherical.lon.value
    data['b'] = c.galactic.spherical.lat.value
    
    print (data['mB'][0])
    print (data['mB'][1])
    
    if out is None:
        out = data
    else:
        for key, value in data.items():
            out[key] = numpy.append(out[key],value)               


/global/u1/a/akim/desc/Peculiar Velocity/buzzard_stellar_mass.py:76: RuntimeWarning: invalid value encountered in true_divide
  met   = np.dot(coeff, self.sfh_tot * self.sfh_met) / ssfr


AttributeError: 'numpy.float32' object has no attribute 'value'

In [ ]:
import pickle
pickle.dump(out, open( "tenyear.{}.pkl".format(seed), "wb" ) )

In [ ]:
sortin = numpy.argsort(out['redshift'])
f = open('pvlist.{}.dat'.format(seed), 'w')
for i in range(len(sortin)):
    print(' '.join(str(e) for e in (out['redshift'][sortin[i]],out['mB'][sortin[i]],0,out['l'][sortin[i]],out['b'][sortin[i]],
                                    out['mB_expected'][sortin[i]], out['nsne'][sortin[i]])),file=f)
f.close()